## Airline Passenger Satisfaction - Data vizualization

Wczytanie pliku testowego oraz trenowanego

In [5]:
import pandas as pd

df_train = pd.read_csv('finalDataset2/train.csv')
df_test = pd.read_csv('finalDataset2/test.csv')

Informacja o brakujących danych w zbiorach

In [6]:
print('Zbiór trenowany:')
print(df_train.isnull().sum())
print()
print('Zbiór testowy:')
print(df_test.isnull().sum())

Zbiór trenowany:
id                                   416
Customer Type                        416
Age                                  416
Type of Travel                       416
Class                                416
Flight Distance                      416
Inflight wifi service                416
Departure/Arrival time convenient    416
Ease of Online booking               416
Gate location                        416
Food and drink                       416
Online boarding                      416
Seat comfort                         416
Inflight entertainment               416
On-board service                     416
Leg room service                     416
Baggage handling                     416
Checkin service                      416
Inflight service                     416
Cleanliness                          416
Departure Delay in Minutes           416
Arrival Delay in Minutes             723
satisfaction                         416
Male                                 416